In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
import utils

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
name_mapping = {
    'INDEX_Z_SPREAD': 'z_spread',
    'CONTRBTD_ZSPREAD': 'z_spread',
    'PX_LAST': 'z_spread'
}

In [ ]:
dt_fmt = '%Y%m%d'
bq = BlpQuery(timeout=20000).start()

In [ ]:
indices_ref = pd.read_csv(data_path / 'indices_reference.csv')
indices_hist = pd.read_csv(data_path / 'indices_history.csv', parse_dates=['date'], dayfirst=True) 

In [ ]:
# excluding iBoxx for now
# indices_ref = indices_ref.loc[indices_ref['fields'] != "['CONTRBTD_ZSPREAD']", :]

In [ ]:
start_date = pd.Timestamp(2007, 1, 1)
end_date = pd.Timestamp.today().date() - BDay()

In [ ]:
g = indices_hist.groupby(['security'])['date'].max()
g

security
BCR5TRUU Index            2022-05-24
BEHLTREU Index            2022-05-24
BHYOTRUU Index            2022-05-24
BLQCTRUU Index            2022-05-24
BUS3TRUU Index            2022-05-24
                             ...    
LUAITRUU Index            2022-05-24
QW5A Index                2022-05-24
QW5M Index                2022-05-24
SNRFIN CDSI GEN 5Y Corp   2022-05-25
SUBFIN CDSI GEN 5Y Corp   2022-05-25
Name: date, Length: 104, dtype: datetime64[ns]

In [ ]:
res = {}

for ticker, last_date in g.items():
    if last_date < end_date:
        row = indices_ref.loc[indices_ref['ticker'] == ticker].squeeze()
        print(f"downloading {ticker} ..")
        res_ = bq.bdh(
            securities=[ticker],
            fields=eval(row['fields']),
            start_date=(last_date + BDay()).strftime(dt_fmt), 
            end_date=end_date.strftime(dt_fmt)
        )
        res_ = res_.rename(name_mapping, axis=1)
        res_['z_spread'] = res_['z_spread'] / row['scale']
        res[ticker] = res_

downloading BCR5TRUU Index ..
downloading BEHLTREU Index ..
downloading BHYOTRUU Index ..
downloading BLQCTRUU Index ..
downloading BUS3TRUU Index ..
downloading CDX HY CDSI GEN 5Y SPRD Corp ..
downloading CDX IG CDSI GEN 5Y Corp ..
downloading H30585EU Index ..
downloading I00154US Index ..
downloading I00155US Index ..
downloading I00159US Index ..
downloading I00160US Index ..
downloading I00161US Index ..
downloading I00182US Index ..
downloading I00408US Index ..
downloading I02008EU Index ..
downloading I02009EU Index ..
downloading I02137EU Index ..
downloading I02138EU Index ..
downloading I02199EU Index ..
downloading I02200EU Index ..
downloading I02201EU Index ..
downloading I02202EU Index ..
downloading I03125EU Index ..
downloading I03139EU Index ..
downloading I04941US Index ..
downloading I04942US Index ..
downloading I04943US Index ..
downloading I04944US Index ..
downloading I04945US Index ..
downloading I04946US Index ..
downloading I04947US Index ..
downloading I0494

In [ ]:
if res:
    new_hist = pd.concat(res.values())
    indices_hist = pd.concat([indices_hist , new_hist])\
                     .drop_duplicates()\
                     .sort_values(['security', 'date'])
    indices_hist.to_csv(data_path / 'indices_history.csv', index=False)